In [1]:
from gurobipy import *

# Data structures useful for modeling optimization problems

### Tuples
Good for multidimensional subscripts

Cannot be modified after creation

In [2]:
arc = ('CHI', 'NYC')

### Lists
Ordered groups. Order will be maintaines

Can be modified.

Be aware of repeated elements

In [3]:
cities=['CHI', 'NYC', 'ATL', 'MIA']

### Dictionary
Representing indexed data

In [6]:
cost = {}
cost[('CHI', 'NYC')] = 100

### Tuplelist
Gurobi extension of storing a list of tuples

In [8]:
arcs = tuplelist([
      ('CHI', 'NYC'), ('CHI', 'ATL'), ('ATL', 'MIA'), ('ATL', 'NYC')  
    ])

the select() method finds matching subsets within the tuplelist, using efficient indexing

In [10]:
print arcs.select('CHI', '*')

[('CHI', 'NYC'), ('CHI', 'ATL')]


### Multidict
A convenience function to initialize dictionaries and their indices at once.

Outputs an index (a list of the keys), and dictionaries mapping key values to the values given in the list.

In [15]:
cities, supply, demand = multidict({
        'ATL': [100,20], 
        'CHI': [150, 50],
        'NYC': [20,300], 
        'MIA': [10, 200]
    })

In [16]:
cities

['MIA', 'CHI', 'ATL', 'NYC']

In [17]:
supply

{'ATL': 100, 'CHI': 150, 'MIA': 10, 'NYC': 20}

In [18]:
demand

{'ATL': 20, 'CHI': 50, 'MIA': 200, 'NYC': 300}

#### quicksum()
The gurobi quicksum function provides a fast way to add up the costs (more efficient than sum()). The statement below corresponds to 

$ \sum_{c\in Cities} supply(c) - demand(c) $


In [19]:
quicksum(supply[c] - demand[c] for c in cities)

<gurobi.LinExpr: -290.0>

## Decision variables

decision variables are represented in python as dictionaries. The keys of the decision variables are the indices which can be single or multidimensional. Multidimensional indices are represented with tuples. The values in the dictionary are then the decision variable objects. 

For example, binary decision variables $x_{ij}$ are represented as:

Decision variables are added to a model via 
ref: <a href="https://www.gurobi.com/documentation/6.5/refman/py_model_addvar.html"> Model.addVar()</a>
Parameters of the constructor are:
<ul>
<li> lb: lower bound
<li> ub: upper bound
<li> obj: objective coefficient (?)
<li> vtype: variable type, GRB.CONTINUOUS, BINARY, INTEGER; SEMICONT, SEMIINT
<li> name
</ul>

In [29]:
m = Model("test")
x = {}
for i in range(0,5):
    for j in range(0,4):        
        x[(i,j)] = m.addVar(vtype=GRB.BINARY, name="x"+str(i)+str(j))
        
#call update() method to explicitly add the variables to the model
m.update()

### Exploit sparsity in modeling:

If you have multidimensional decision variables $x_{ij}$ with only few potentially valid assignments $ i \rightarrow j$ then you can exploit this for faster performance (can have a huge impact!)
<ol>
<li> Use tuplelist to store the valid combinations
    <code>
    valid = tuplelist([('i1', 'j5'), ('i1', 'j7'), 
                        ('i2', 'j1'), ('i2', 'j7'), ...])</code>
<li> then only create decision variable for valid combinations
    <code> 
    x={}
    for i,j in valid:
        x[i,j] = m.addVar(vtype=GRB.BINARY)
    </code>
<li> Use the select function to efficiently iterate over valid combinations. For example for adding constraints. Only add constraints for valid combinations, using the select function:
<code>
    constraints = {}
    for j in J:
        constraint[j] = m.addConstr(
            requirement[j] == quicksum(x[i,j] for i,j 
                                    in valid.select('*',j) )
            )
</code>
</ol>

# Implementation recommendations

### Model / data separation in Python: 

Write the model inside a function and put to a separate file (module). Function takes as argument all input values needed and then handles the model creation, adding variables, constraints etc. Also allows to easily change the model, using the same input values, e.g. adding additional constraints.
Data can come from any source, e.g. database, excel files, pandas dataframes etc. 

###  Model.update()
Only call update function when necessary, to refrence new objects - variables, constraints, etc. That is, first create all variables and then call update. Then create constraints and then call update again.

# A simple example:
Many examples available in gurobi installation folder, under "examples/python"

In [25]:
# Create a new model
m = Model("mip1")

# Create variables
x = m.addVar(vtype=GRB.BINARY, name="x")
y = m.addVar(vtype=GRB.BINARY, name="y")
z = m.addVar(vtype=GRB.BINARY, name="z")

# Integrate new variables
m.update()

# Set objective
m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

# Add constraint: x + 2 y + 3 z <= 4
m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

# Add constraint: x + y >= 1
m.addConstr(x + y >= 1, "c1")

m.optimize()

for v in m.getVars():
    print('%s %g' % (v.varName, v.x))

print('Obj: %g' % m.objVal)

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 3e+00]
  Objective range [1e+00, 2e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 4e+00]
Found heuristic solution: objective 2
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.48 seconds
Thread count was 1 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0%
x 1
y 0
z 1
Obj: 3
